In [50]:
from tika import parser
from os import listdir
from os.path import isfile, join
import re
from collections import defaultdict

day = "20180223"
path = "source-pdf/"

In [51]:
def retriveFiles(path):
    return [join(path, f) for f in listdir(path) if isfile(join(path, f))]

In [52]:
files = retriveFiles(path)

In [53]:
import time
from datetime import date

In [54]:
def generateDates(files):
    root = len(path)
    res = []
    for file in files:
        dstr = file[root:root+8]
        d = date(int(dstr[0:4]),int(dstr[4:6]),int(dstr[6:8]))
        res.append(d)
    return res

def getDateObject(s):
    return date(int(s[0:4]),int(s[5:7]),int(s[8:10]))

In [55]:
days = generateDates(files)

In [56]:
def retrieveRatings(files, day):
    for file in files:
        if day in file:
            break
    raw = parser.from_file(file)
    pattern = '\(.+\) Q+'
    data_pairs = re.findall(pattern, raw['content'])
    res = defaultdict(int)
    for data in data_pairs:
        pair = data.split(' ');
        res[pair[0][1:-1]] = len(pair[1])
        
    return res
    

In [93]:
ticker_rating_pair = retrieveRatings(files, day)

In [58]:
from tkinter import *
import tkinter as ttk
 
root = Tk()
root.title("Morningstar Stimulator")
 
# Add a grid
mainframe = Frame(root)
mainframe.grid(column=0,row=0, sticky=(N,W,E,S) )
mainframe.columnconfigure(0, weight = 1)
mainframe.rowconfigure(0, weight = 1)
mainframe.pack(pady = 100, padx = 100)
 
# Create a Tkinter variable
tkvar = StringVar(root)
 
# Dictionary with options
choices = days
tkvar.set('') # set the default option
 
popupMenu = OptionMenu(mainframe, tkvar, *choices)
Label(mainframe, text="Choose Date").grid(row = 1, column = 1)
popupMenu.grid(row = 2, column =1)
 
# on change dropdown value
def change_dropdown(*args):
    tkvar.set(tkvar.get())
    print( getDateObject(tkvar.get()).strftime('We are the %d, %b %Y') )
 
# link function to change dropdown
tkvar.trace('w', change_dropdown)
 
root.mainloop()

In [59]:
from pandas_datareader import data
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime

In [89]:
# Define the instruments to download. We would like to see Apple, Microsoft and the S&P500 index.
tickers = ['0700', 'MSFT', '^GSPC']

# We would like all available data from 01/01/2000 until 12/31/2016.
report_date = datetime(2018, 3, 7)
end_date = datetime(2018, 3, 15)

# User pandas_reader.data.DataReader to load the desired data. As simple as that.
panel_data = data.DataReader('MSFT', 'yahoo', report_date, end_date)

for ticker in tickers:
    try:    
        print(ticker, data.DataReader(ticker, 'yahoo', report_date, end_date).iloc[0]['Close'], sep = ": $")
    except:
        print(ticker, " is not public traded in North America stock market, ignored...", sep = "")




0700 is not public traded in North America stock market, ignored...
MSFT: $93.86000061035156
^GSPC: $2726.800048828125


In [90]:
panel_data

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2018-03-07,93.940002,92.430000,93.160004,93.860001,26716100,92.302025
2018-03-08,95.099998,93.769997,94.269997,94.430000,25887800,92.862556
2018-03-09,96.540001,95.000000,95.290001,96.540001,36937300,94.937538
2018-03-12,97.209999,96.040001,96.500000,96.769997,26073700,95.163712
2018-03-13,97.239998,93.970001,97.000000,94.410004,35387800,92.842896
2018-03-14,95.410004,93.500000,95.120003,93.849998,32132000,92.292183
2018-03-15,94.580002,92.830002,93.529999,94.180000,27611000,92.616714
2018-03-16,95.379997,93.919998,94.680000,94.599998,49081300,93.029739
2018-03-19,93.900002,92.110001,93.739998,92.889999,33344100,91.348114


In [104]:
#get open and close prices from yahoo api
#return dic {ticker: [rating, [price1, price2, ...]], ...}
def getOpenClosePrices(report_date, end_date, ticker_rating_pair):
    res = defaultdict(list)
    for ticker, rating in ticker_rating_pair.items():
        try:    
            price_info = data.DataReader(ticker, 'yahoo', report_date, end_date)
            prices = []
            for i in range (price_info.shape[0]):
                prices.append(price_info.iloc[0]['Close'])
            res[ticker] = [rating, prices]
        except:
            print(ticker, " is not public traded in North America stock market, ignored...", sep = "")
    return res

In [107]:
ticker_rating_pair = {'0700': 3, 'MSFT': 3, '^GSPC': 3}


In [110]:
ticker_prices = getOpenClosePrices(report_date, end_date, ticker_rating_pair)

0700 is not public traded in North America stock market, ignored...


In [ ]:
#calculate daily profits
#return list [profit1, profit2]
#prices: dict {ticker: [rating, [price1, price2, ...]], ...}
#shares: dict {3: quantity, 4: quantity, 5: quantity}
shares = [3: 300, 4: 400, 5: 500]
def calculateProfits(ticker_prices, shares):
    profits  = []
    for ticker, ops in ticker_prices.items:
        rating = ops[0]
        prices = ops[1]
        for i in range(len(prices)):
            profits[i] = (prices[i+1] - prices[0])*shares[rating]